In [306]:
import torch
from torchvision import transforms, datasets
import torch.utils.data as data
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [307]:
df = pd.read_csv("train_data.csv")

# Data processing
- y - class from 0 to 3 according to the following:
    - 0 to 100k - class 0
    - 100k to 350k - class 1
    - 350k and above - class 2
        

Features:
- YearBuilt - numeric
- Size(sqf) - numeric
- Floor - numeric
- HallwayType - one hot encoding
- HeatingType - binary encoding
- AptManageType - binary encoding
- N_Parkinglot(Ground) - numeric
- N_Parkinglot(Basement) - numeric
- TimeToBusStop - one hot encoding
- TimeToSubway - one hot encoding
- N_manager - numeric
- N_elevators - numeric
- SubwayStation - one hot encoding
- N_FacilitiesInApt - numeric
- N_FacilitiesNearBy(Total) - numeric
- N_SchoolNearBy(Total) - numeric

Train data: 4124
Train data per class: 0 - 562, 1 - 2992, 2 - 570


In [308]:
df

,SalePrice,YearBuilt,Size(sqf),Floor,HallwayType,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_manager,N_elevators,SubwayStation,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,141592,2006,814,3,terraced,individual_heating,management_in_trust,111.0,184.0,5min~10min,10min~15min,3.0,0.0,Kyungbuk_uni_hospital,5,6.0,9.0
1,51327,1985,587,8,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
2,48672,1985,587,6,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
3,380530,2006,2056,8,terraced,individual_heating,management_in_trust,249.0,536.0,0~5min,0-5min,5.0,11.0,Sin-nam,5,3.0,7.0
4,78318,1992,644,2,mixed,individual_heating,self_management,142.0,79.0,5min~10min,15min~20min,4.0,8.0,Myung-duk,3,9.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,570796,2007,1928,24,terraced,individual_heating,management_in_trust,0.0,1270.0,0~5min,0-5min,14.0,16.0,Kyungbuk_uni_hospital,10,9.0,10.0
4120,307079,2015,644,22,terraced,individual_heating,management_in_trust,102.0,400.0,0~5min,5min~10min,5.0,10.0,Daegu,7,7.0,11.0
4121,357522,2007,868,20,terraced,individual_heating,management_in_trust,0.0,1270.0,0~5min,0-5min,14.0,16.0,Kyungbuk_uni_hospital,10,9.0,10.0
4122,312389,1978,1327,1,corridor,individual_heating,self_management,87.0,0.0,0~5min,0-5min,1.0,4.0,Kyungbuk_uni_hospital,3,7.0,11.0


In [309]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    # Add classes
    df['class'] = pd.cut(df['SalePrice'], bins=[-float('inf'), 100000, 350000, float('inf')], labels=[0, 1, 2])
    df = df.drop(columns=["SalePrice"])
    
    # One hot encoding
    for_one_hot_encoding = ['HallwayType', 'TimeToBusStop', 'TimeToSubway', 'SubwayStation']
    drop_first = False
    
    df = pd.get_dummies(df, columns=for_one_hot_encoding, drop_first=drop_first)
    
    # Binary encoding
    for_binary = ['HeatingType', 'AptManageType']
    df = pd.get_dummies(df, columns=for_binary, drop_first=True)
    
    # Scale numeric to range 0 to 1
    to_be_scaled = ['YearBuilt', 'Size(sqf)', 'Floor', 'N_Parkinglot(Ground)', 'N_Parkinglot(Basement)', 'N_manager', 'N_elevators', 'N_FacilitiesInApt', 'N_FacilitiesNearBy(Total)', 'N_SchoolNearBy(Total)']
    for column in to_be_scaled:
        df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    
    return df

In [310]:
df = preprocess_data(df)
df

,YearBuilt,Size(sqf),Floor,N_Parkinglot(Ground),N_Parkinglot(Basement),N_manager,N_elevators,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total),...,SubwayStation_Bangoge,SubwayStation_Banwoldang,SubwayStation_Chil-sung-market,SubwayStation_Daegu,SubwayStation_Kyungbuk_uni_hospital,SubwayStation_Myung-duk,SubwayStation_Sin-nam,SubwayStation_no_subway_nearby,HeatingType_individual_heating,AptManageType_self_management
0,0.756757,0.308356,0.047619,0.155680,0.139288,0.153846,0.000000,0.444444,0.3750,0.529412,...,False,False,False,False,True,False,False,False,True,False
1,0.189189,0.205268,0.166667,0.112202,0.057532,0.076923,0.074074,0.222222,0.7500,0.235294,...,False,False,False,True,False,False,False,False,True,True
2,0.189189,0.205268,0.119048,0.112202,0.057532,0.076923,0.074074,0.222222,0.7500,0.235294,...,False,False,False,True,False,False,False,False,True,True
3,0.756757,0.872389,0.166667,0.349229,0.405753,0.307692,0.407407,0.444444,0.1875,0.411765,...,False,False,False,False,False,False,True,False,True,False
4,0.378378,0.231153,0.023810,0.199158,0.059803,0.230769,0.296296,0.222222,0.5625,0.823529,...,False,False,False,False,False,True,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,0.783784,0.814260,0.547619,0.000000,0.961393,1.000000,0.592593,1.000000,0.5625,0.588235,...,False,False,False,False,True,False,False,False,True,False
4120,1.000000,0.231153,0.500000,0.143058,0.302801,0.307692,0.370370,0.666667,0.4375,0.647059,...,False,False,False,True,False,False,False,False,True,False
4121,0.783784,0.332879,0.452381,0.000000,0.961393,1.000000,0.592593,1.000000,0.5625,0.588235,...,False,False,False,False,True,False,False,False,True,False
4122,0.000000,0.541326,0.000000,0.122020,0.000000,0.000000,0.148148,0.222222,0.4375,0.647059,...,False,False,False,False,True,False,False,False,True,True


In [311]:
df.shape

(4124, 32)

In [312]:
train_dataset_raw = df.sample(frac=0.8, random_state=200)
df = df.drop(train_dataset_raw.index)
val_dataset_raw = train_dataset_raw.sample(frac=0.125, random_state=200) # 10% of original dataset
train_dataset_raw = train_dataset_raw.drop(val_dataset_raw.index)

test_dataset_raw = df
print("Dataset sizes")
print(f"Train: {len(train_dataset_raw)} samples")
print(f"Val:   {len(val_dataset_raw)} samples")
print(f"Test:  {len(test_dataset_raw)} samples")

Dataset sizes
Train: 2887 samples
Val:   412 samples
Test:  825 samples


In [313]:
train_labels = train_dataset_raw['class'].to_numpy().astype(np.int64)
train_data = train_dataset_raw.drop(columns=['class']).to_numpy().astype(np.float32)

val_labels = val_dataset_raw['class'].to_numpy().astype(np.int64)
val_data = val_dataset_raw.drop(columns=['class']).to_numpy().astype(np.float32)

test_labels = test_dataset_raw['class'].to_numpy().astype(np.int64)
test_data = test_dataset_raw.drop(columns=['class']).to_numpy().astype(np.float32)

In [314]:
train_dataset = data.TensorDataset(torch.from_numpy(train_data), torch.from_numpy(train_labels))
val_dataset = data.TensorDataset(torch.from_numpy(val_data), torch.from_numpy(val_labels))
test_dataset = data.TensorDataset(torch.from_numpy(test_data), torch.from_numpy(test_labels))

In [315]:
BATCH_SIZE = 256
train_data_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_data_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Model

In [316]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(device))
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

else:
    device = torch.device("cpu")

print(device.type)

NVIDIA GeForce RTX 4080 SUPER
cuda


In [317]:
class ApartmentClassifier(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.input = nn.Linear(num_inputs, 512)
        self.hidden1 = nn.Linear(512, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, num_outputs)
        self.act_fn = nn.LeakyReLU()

    def forward(self, x):
        x = self.input(x)
        x = self.act_fn(x)
        x = self.hidden1(x)
        x = self.act_fn(x)
        x = self.hidden2(x)
        x = self.act_fn(x)
        x = self.output(x)
        return x


In [318]:
model = ApartmentClassifier(31, 3)
model.to(device)

ApartmentClassifier(
  (input): Linear(in_features=31, out_features=512, bias=True)
  (hidden1): Linear(in_features=512, out_features=128, bias=True)
  (hidden2): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=3, bias=True)
  (act_fn): LeakyReLU(negative_slope=0.01)
)

In [319]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_module = nn.CrossEntropyLoss()

In [320]:
EPOCHS = 100
train_losses = []
val_losses = []
for epoch in range(EPOCHS):
    total_train_loss = 0
    total_val_loss = 0
    
    train_correct = 0
    train_total = 0
    
    val_correct = 0
    val_total = 0

    model.train()
    for data_inputs, data_labels in train_data_loader:
        data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)

        prediction = model(data_inputs)
        prediction = prediction.squeeze(dim=1)

        loss = loss_module(prediction, data_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        
        preds = prediction.argmax(1)
        train_correct += (preds == data_labels).sum().item()
        train_total += data_labels.size(0)

    train_accuracy = 100 * train_correct / train_total
    avg_train_loss = total_train_loss / len(train_data_loader)


    model.eval()
    with torch.no_grad():
        for data_inputs, data_labels in val_data_loader:
            data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)

            prediction = model(data_inputs)
            prediction = prediction.squeeze(dim=1)

            loss = loss_module(prediction, data_labels)
            total_val_loss += loss.item()
            
            preds = prediction.argmax(1)
            val_correct += (preds == data_labels).sum().item()
            val_total += data_labels.size(0)
    
    val_accuracy = 100 * val_correct / val_total
    avg_val_loss = total_val_loss / len(val_data_loader)

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    print(f"Epoch {epoch+1}| Train Loss = {avg_train_loss:.4f} Train acc = {train_accuracy:.4f}| Val Loss = {avg_val_loss:.4f}| Val acc = {val_accuracy:.4f}")

Epoch 1| Train Loss = 0.8623 Train acc = 72.6360| Val Loss = 0.7356| Val acc = 73.3010
Epoch 2| Train Loss = 0.6544 Train acc = 73.3287| Val Loss = 0.5893| Val acc = 73.5437
Epoch 3| Train Loss = 0.5219 Train acc = 76.4461| Val Loss = 0.4912| Val acc = 75.4854
Epoch 4| Train Loss = 0.4412 Train acc = 80.4641| Val Loss = 0.4552| Val acc = 80.3398
Epoch 5| Train Loss = 0.4077 Train acc = 83.2352| Val Loss = 0.4307| Val acc = 82.2816
Epoch 6| Train Loss = 0.3934 Train acc = 84.1011| Val Loss = 0.4337| Val acc = 79.8544
Epoch 7| Train Loss = 0.3872 Train acc = 83.4777| Val Loss = 0.4151| Val acc = 81.7961
Epoch 8| Train Loss = 0.3807 Train acc = 84.3090| Val Loss = 0.4105| Val acc = 82.0388
Epoch 9| Train Loss = 0.3635 Train acc = 84.3436| Val Loss = 0.3696| Val acc = 81.7961
Epoch 10| Train Loss = 0.3637 Train acc = 84.2743| Val Loss = 0.3720| Val acc = 83.4951
Epoch 11| Train Loss = 0.3558 Train acc = 84.5861| Val Loss = 0.3647| Val acc = 81.7961
Epoch 12| Train Loss = 0.3402 Train acc =

In [321]:
model.eval() # Set model to eval mode
true_preds, num_preds = 0., 0.

test_y_hat = []
test_y = []
with torch.no_grad(): # Deactivate gradients for the following code
    for data_inputs, data_labels in test_data_loader:

        # Determine prediction of model on dev set
        data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)
        logits = model(data_inputs)
        pred_probabilites = nn.Softmax(dim=1)(logits)
        pred_labels = pred_probabilites.argmax(1)
        test_y_hat.append(pred_labels.to("cpu").numpy())
        test_y.append(data_labels.to("cpu").numpy())

        true_preds += (pred_labels == data_labels).sum()
        num_preds += data_labels.shape[0]

acc = true_preds / num_preds
print(f"Accuracy of the model: {100.0*acc:4.2f}%")

Accuracy of the model: 86.42%
